In [ ]:
#Naive Bayes Computer Assignment 02
#This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.

In [8]:
import os as os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
#"""This function builds a Dictionary of most common 3000 words from all the email content. 
#First it adds all words and symbols in the dictionary. Then it removes all non-alpha-numeric characters 
#and any single character alpha-numeric characters. After this is complete it shrinks the Dictionary 
#by keeping only most common 3000 words in the dictionary. It returns the Dictionary."""

In [6]:
def Data_Dictionary(root_dir):
  data_words = []
  #this OS function is used to create a list of individual paths to each data file
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
  for email in emails:
    with open(email) as m:
      for line in m:
        words = line.split()
        data_words += words
  print("all file words have been read and saved")
  #This line creates a counter object which is a sub object of dictionary to hold counts of each word
  dictionary = Counter(data_words)
  #This line makes the dictionary iteratable to be cleaned
  uncleaned_list = list(dictionary)
  
  #This for loop goes through the entire set of words in the data and removes those that don't contain only
  # letters a-z or are words that are 1 character long such as "a"
  for item in uncleaned_list:
    if item.isalpha() == False:
      del dictionary[item]
    elif len(item) == 1:
      del dictionary[item]
  print("numeric strings and one character words have been removed")
  dictionary = dictionary.most_common(3000)
  print("Dictionary has been reduced to 3000 words")
  return dictionary

In [ ]:
#This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to 
#the number of email files). The function also analyzes the File Names of each email file and 
#decides if it's a spam or not based on the file name. Based on this the function also creates
#the Labelled Data Column. This function is used to extract the training dataset 
#as well as the testing dataset and returns the Feature Dataset and the Label column.

In [18]:
def get_features(mail_dir):
  #this OS method combines file directory with the folder directory to get a list of file paths
  emails = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(emails),3000)) #create blank matrix with 3000 columns and length of # of emails
  train_labels = np.zeros(len(emails)) #creates empty array of length # of emails for spam labels
  print("empty matrix and array created")
  count = 1;
  docID = 0;
  for fil in emails:
    with open(fil) as fi: #opens each email file in the folder provided
      for i, line in enumerate(fi):
        if i ==2: #this starts the parsining at the 2nd line
          words = line.split() #parses each line of the file
          for word in words: #loops through the words in each line
            wID = 0
            for i, d in enumerate(dictionary): #loops through words in created training dictionary
              if d[0] == word: #checks if word matches dictionary entry
                wID = i
                features_matrix[docID,wID] = words.count(word) #adds count of words for that word in matrix for that email
      train_labels[docID] = 0; #array equals 0 if no match
      filepathTokens = fil.split('/') #splits file name
      lastToken = filepathTokens[len(filepathTokens)-1] #saves last part of file name
      if lastToken.startswith("spmsg"): 
        train_labels[docID] = 1; #assigns 1 to document ID if email labeled spam
        count = count + 1
      docID = docID + 1 #incrememnts ID index tracker
  print(count)
  print("all emails looped through, assigned to ")
  return features_matrix, train_labels

In [ ]:
#Main program which brings in data sets of emails then processes them using the above functions.
#Below is the call which creates the dictionary

In [14]:
#Create partial paths for Test and Train data
TRAIN_DIR = 'Data/train-mails'
TEST_DIR = 'Data/test-mails'

dictionary = Data_Dictionary(TRAIN_DIR)
len(dictionary) #Test view of dictionary
dictionary[0] #test dictionary

('order', 1414)

In [20]:
#This cell uses the above function to pull the word count matrix as well as the training lables for Train and Test data
#the training labels hold the classifications for spam or not spam (1 or 0)
print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, train_labels = get_features(TRAIN_DIR)
test_features_matrix, test_labels = get_features(TEST_DIR)


reading and processing emails from TRAIN and TEST folders
empty matrix and array created
352
all emails looped through, assigned to 
empty matrix and array created
131
all emails looped through, assigned to 


In [30]:
#This section initiates a Naive Bayes model then uses the training emails to train and produces an accuracy score
#based on the test emails
model = GaussianNB()

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
model.fit(features_matrix, train_labels)
print ("Training completed")
print ("testing trained model to predict Test Data labels")
predicted_labels = model.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print ("Score equals" ,accuracy_score(test_labels, predicted_labels))
print("Predicted sample:" , predicted_labels[0:10])
print("Actual sample:", test_labels[0:10])


Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:
Score equals 0.9653846153846154
Predicted sample: [1. 0. 0. 0. 1. 1. 1. 0. 1. 0.]
Actual sample: [1. 0. 0. 0. 1. 1. 1. 0. 1. 0.]
